# Getting Started with OpenQASM on Braket

OpenQASM is a popular human-readable and hardware-agnostic quantum circuit description language. It is open-source and has been actively maintained by a [Technical Steering Committee](https://medium.com/qiskit/introducing-a-technical-steering-committee-for-openqasm3-f9db808108e1) formed by IBM, Amazon, Microsoft and the University of Innsbruck. Amazon Braket now supports OpenQASM 3.0 as an *Intermediate Representation* (IR) in addition to the in-house *JSON-Based AWS Quantum Circuit Description* ([JAQCD](https://github.com/aws/amazon-braket-schemas-python/tree/main/src/braket/ir/jaqcd)). In this notebook, we demonstrate how to submit OpenQASM tasks to various devices on Braket and introduce some OpenQASM features available on Braket.

## Create and submit an OpenQASM task

Submitting a quantum task with OpenQASM is just as simple as using JAQCD. You can use the Amazon Braket Python SDK, Boto3, or the AWS CLI to submit OpenQASM 3.0 tasks to an Amazon Braket device. We will go over each method in this section.


### A Bell state

We will start with by preparing a [Bell state](https://en.wikipedia.org/wiki/Bell_state) in OpenQASM:


In [6]:
bell_qasm = """
OPENQASM 3;

qubit[2] q;
bit[2] c;

h q[0];
cnot q[0], q[1];

c = measure q;
"""

Compare this to the same Bell state written in JAQCD:

In [7]:
from braket.ir.jaqcd import CNot, H, Program

program = Program(instructions=[H(target=0), CNot(control=0, target=1)])
print(program.json(indent=2))

{
  "braketSchemaHeader": {
    "name": "braket.ir.jaqcd.program",
    "version": "1"
  },
  "instructions": [
    {
      "target": 0,
      "type": "h"
    },
    {
      "control": 0,
      "target": 1,
      "type": "cnot"
    }
  ],
  "results": null,
  "basis_rotation_instructions": null
}


Immediately, we can see a difference: In OpenQASM, users define their own qubit registers, and thus the syntax is closer to what quantum algorithm researchers are used to; on the other hand, in JAQCD, qubits are indexed by integers and the convention is closer to that of hardware providers. Also, JAQCD has result types and basis rotation instructions embedded in the language while OpenQASM doesn't support them inherently (but later we will show how to use the `pragma` syntax to support them in OpenQASM).


### Use the Python SDK to create OpenQASM 3.0 tasks

Most Braket users might want to use the Braket Python SDK to submit OpenQASM tasks. To submit our Bell state program in the Python SDK, we first choose the quantum device that we want to run our  program on. In this example, we will use the SV1 state-vector simulator for demonstration.

In [8]:
import boto3
import json
from braket.aws import AwsDevice

To submit the OpenQASM task, we initialize an `OpenQASMProgram` object using the Bell state program text string `bell_qasm` we defined above and send it to the SV1 simulator.

In [12]:
from braket.ir.openqasm import Program as OpenQASMProgram

sv1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

bell_program = OpenQASMProgram(source=bell_qasm)

bell_task = sv1.run(
    bell_program, 
    shots=100, 
)

###  Submit OpenQASM 3.0 programs using the AWS Command Line Interface

Alternatively, if you like the command line experience or you are not a Python user, you can also choose to use the [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/) to submit our Bell state program. Before doing that we have to make sure we have [AWS CLI installed](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html). The following code saves the `bell_qasm` string to a file named `bell.qasm`:

In [91]:
with open("bell.qasm", "w") as f:
    f.write(bell_qasm)

Then we can use the command below to submit the task via AWS CLI. Remember to replace the placeholder \"amazon-braket-my-bucket\" with your own bucket name.
      
    aws braket create-quantum-task \
    --region "us-west-1" \
    --device-arn "arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1" \
    --shots 100 \
    --action '{
        "braketSchemaHeader": {
            "name": "braket.ir.openqasm.program", 
            "version": "1"
        },
        "source": $(cat bell.qasm)
    }'

## Figure out what OpenQASM features are supported on each device

Different devices on Braket support different subsets of OpenQASM features. To see what are the supported OpenQASM features on each device, we can simply check the device capability for OpenQASM actions. As an example, we can take a look at the `action` field in the device capability of SV1 simulator:

In [92]:
# supportedOperations
sv1.properties.action['braket.ir.openqasm.program'].supportedOperations

OpenQASMDeviceActionProperties(version=['1.0'], actionType='braket.ir.openqasm.program', supportedOperations=['ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'cy', 'cz', 'h', 'i', 'iswap', 'pswap', 'phaseshift', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'v', 'vi', 'x', 'xx', 'xy', 'y', 'yy', 'z', 'zz'], supportedPragmas=['braket_unitary_matrix', 'braket_basis_rotation', 'braket_result_type_sample', 'braket_result_type_expectation', 'braket_result_type_variance', 'braket_result_type_probability', 'braket_result_type_amplitude'], forbiddenPragmas=['braket_result_type_state_vector', 'braket_result_type_density_matrix', 'braket_noise_amplitude_damping', 'braket_noise_bit_flip', 'braket_noise_depolarizing', 'braket_noise_kraus', 'braket_noise_pauli_channel', 'braket_noise_generalized_amplitude_damping', 'braket_noise_phase_flip', 'braket_noise_phase_damping', 'braket_noise_two_qubit_dephasing', 'braket_noise_two_qubit_depolarizing'], maximum

In [ ]:
# supportedPragmas
sv1.properties.action['braket.ir.openqasm.program'].supportedPragmas

In [ ]:
# forbiddenPragmas
sv1.properties.action['braket.ir.openqasm.program'].forbiddenPragmas

The SV1 OpenQASM `action` field lists supported/forbidden OpenQASM features on the device, including `supportedPragmas`, `forbiddenPragmas`, `maximumQubitArrays`, `maximumClassicalArrays`, `requiresAllQubitsMeasurement`, `supportedResultTypes`, etc. The names are self-evident, but readers are encouraged to visit the [Amazon Braket developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-using.html) for full information of what these fields mean.

# OpenQASM features on Braket

Braket supports many useful OpenQASM features, either through the OpenQASM program syntax or Braket-specific pragmas. We will walk through some of these features in this section.

## Simulating Noise with OpenQASM

With the fully managed, high-performance, density-matrix simulator [DM1](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-dm1), you can easily investigate the effects of realistic noise on your quantum programs. Now, we show how to use OpenQASM programs to leverage the circuit-level noise simulation capability of DM1.

To simulate noise, we have to be able to specify different noise channels. Although syntax for noise channels is not available in the OpenQASM language, Braket uses the `pragma` statement to extend OpenQASM for defining noise channels. Here is an example of an OpenQASM program that prepares a noisy 3-qubit [GHZ state](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state):

In [93]:
noisy_ghz3_program = """
// noisy_ghz3.qasm
// Prepare a 3 noisy qubit GHZ state
OPENQASM 3;

qubit[3] q;
bit[3] c;

h q[0];
#pragma braket noise depolarizing(0.1) q[0]
cnot q[0], q[1];
#pragma braket noise depolarizing(0.1) q[0]
#pragma braket noise depolarizing(0.1) q[1]
cnot q[1], q[2];
#pragma braket noise depolarizing(0.1) q[0]
#pragma braket noise depolarizing(0.1) q[1]

c = measure q;
"""

In the example above, we inserted the depolarizing noise channel with probability of 0.1 after each gate in the circuit. The `noisy_ghz3_program` is equivalent to the following program in the Braket SDK:

In [94]:
from braket.circuits import Circuit, Observable, Gate, Noise

noisy_ghz3_circ = Circuit().h(0).cnot(0, 1).cnot(1, 2)
noise = Noise.Depolarizing(probability=0.1)
noisy_ghz3_circ.apply_gate_noise(noise)

Circuit('instructions': [Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': Depolarizing('probability': 0.1, 'qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)])), Instruction('operator': Depolarizing('probability': 0.1, 'qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': Depolarizing('probability': 0.1, 'qubit_count': 1), 'target': QubitSet([Qubit(1)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(1), Qubit(2)])), Instruction('operator': Depolarizing('probability': 0.1, 'qubit_count': 1), 'target': QubitSet([Qubit(1)])), Instruction('operator': Depolarizing('probability': 0.1, 'qubit_count': 1), 'target': QubitSet([Qubit(2)]))])

To see if `noisy_ghz3_program` and `noisy_ghz3_circ` are indeed the same, we can run both circuits and compare the results:

In [95]:
dm1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")
noisy_ghz3_circ_task = dm1.run(noisy_ghz3_circ, shots = 10)
noisy_ghz3_program_task = dm1.run(OpenQASMProgram(source=noisy_ghz3_program), shots = 10)

In [96]:
sdk_result = noisy_ghz3_circ_task.result()
openqasm_result = noisy_ghz3_program_task.result()
sdk_measurement = sdk_result.measurement_counts
openqasm_measurement = openqasm_result.measurement_counts
print('sdk measurement results:', sdk_measurement)
print('openqasm measurement results:', openqasm_measurement)

sdk measurement results: Counter({'111': 4, '000': 3, '011': 2, '001': 1})
openqasm measurement results: Counter({'111': 5, '011': 2, '000': 2, '100': 1})


As expected, the measurement counts of the two are very close.

In addition to depolarizing noises, we can simulate more complicated noise types with Braket, e.g., `pauli_channel`, `amplitude_damping`, etc. Check the [Amazon Braket developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-using.html) for a complete list of noise channels supported on Braket. Here we give another example of general noise channels defined by the Kraus representation.

In [97]:
noisy_program_with_kraus_operators = """
// noisy_program_with_kraus_operators
OPENQASM 3;

qubit[2] q;
bit[2] c;

h q[0];
#pragma braket noise kraus([[0.9486833, 0], [0, 0.9486833]], [[0, 0.31622777], [0.31622777, 0]]) q[0]
cnot q[1], q[2];

c = measure q;
"""

We inserted a single qubit noise channel defined by two 2x2 complex Kraus operators in the example above on qubit `q[0]`. Braket will validate if the Kraus operators indeed form a Completely-Positive and Trace-Preserving (CPTP) map.

## Simulating arbitrary unitaries with OpenQASM

The fully-managed [SV1 simulator](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-sv1) allows us to simulate arbitrary unitary gates in a circuit. With OpenQASM, we can use the `unitary` pramga to insert these arbitrary unitary gates: 

In [98]:
program_with_unitary = """
// noisy_program_with_kraus_operators
OPENQASM 3;

qubit q;
bit c;

#pragma braket unitary([[0, -1im], [1im, 0]]) q
c = measure q;
"""

The `1im` in the `unitary` pragma is the OpenQASM notation of the imaginary number $i$, thus, we were simply using the pragma to perform a Pauli Y gate. We can check it by submitting the above program to SV1.

In [99]:
unitary_task = dm1.run(OpenQASMProgram(source=program_with_unitary), shots = 10)
unitary_result = unitary_task.result()
print(unitary_result.measurement_counts)

Counter({'1': 10})


As expected, the Pauli Y gate flipped the initial 0 state to the 1 state.

## Qubit Rewiring with OpenQASM

Amazon Braket supports the [physical qubit notation within OpenQASM](https://qiskit.github.io/openqasm/language/types.html#physical-qubits) on Rigetti devices. When using physical qubits, you have to ensure that the qubits are indeed connected on the selected device. Alternatively, if qubit registers are used instead, the `PARTIAL` rewiring strategy is enabled by default on Rigetti devices. The following example shows how to use physical qubit notation in an OpenQASM program:

In [110]:
ghz_program_with_physical_qubits = """
// Prepare a GHZ state
OPENQASM 3;

bit[3] ro;
h $0;
cnot $0, $1;
cnot $1, $2;
ro[0] = measure $0;
ro[1] = measure $1;
ro[2] = measure $2;
"""

We can run the above program on the Rigetti Aspen-M-1 device,

In [111]:
# choose the quantum device
aspen_m1 = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")

ghz_program_with_physical_qubits_task = aspen_m1.run(OpenQASMProgram(source=ghz_program_with_physical_qubits), shots = 10)
measured_qubits = ghz_program_with_physical_qubits_task.result().measured_qubits
print("Measured qubits:", measured_qubits)

Measured qubits: [0, 1, 2]


As we can see, physical qubits 0, 1 and 2 are indeed being used and measured.

<div class="alert alert-block alert-info">
    <b>Note:</b> This section and the next verbatim box section uses the Rigetti Aspen-M-1 device. When you run this notebook, make sure the device is currently available. You can find QPU availability windows on the <a href="https://us-west-1.console.aws.amazon.com/braket/home?region=us-west-1#/devices">Devices page</a> in the Amazon Braket Console
</div>

## Verbatim Compilation with OpenQASM

In [a previous example notebook](https://github.com/aws/amazon-braket-examples/blob/main/examples/braket_features/Verbatim_Compilation.ipynb), we talked about verbatim compilation on Braket to gain more precise control on Rigetti devices. With OpenQASM3.0, we can use the `box` syntax together with the `verbatim` pragma to perform verbatim compilation. Here is an example:

In [133]:
program_with_verbatim_box = """
OPENQASM 3;

bit[2] ro;
#pragma braket verbatim
box{
 rx(3.141592653589793) $0;
 rx(3.141592653589793) $0;
 cz $0, $1;
}
ro[0] = measure $0;
ro[1] = measure $1;
"""

To program with verbatim boxes, we need to make sure that
- we are using native gates supported by Rigetti devices. Native gates can be found using the following script:


In [134]:
print("The native gates for the", aspen_m1.name, "device are:")
for gate in aspen_m1.properties.paradigm.nativeGateSet:
    print(gate)

The native gates for the Aspen-11 device are:
rx
rz
cz
cphaseshift
xy


- we use the physical qubit notation.
- qubit operands are indeed connected on the physical device. Recall that the device qubit connectivity can be found using the following commands:

In [135]:
import networkx as nx
# access and visualize the device topology
print(aspen_m1.properties.paradigm.connectivity.connectivityGraph)
nx.draw_kamada_kawai(aspen_m1.topology_graph, with_labels=True, font_color="white")

{'0': ['1', '7'], '1': ['0', '16', '2'], '10': ['11', '17'], '11': ['10', '12', '26'], '12': ['11', '13', '25'], '13': ['12', '14'], '14': ['13', '15'], '15': ['14', '16', '2'], '16': ['1', '15', '17'], '17': ['10', '16'], '2': ['1', '15', '3'], '20': ['21', '27'], '21': ['20', '22', '36'], '22': ['21'], '23': ['24'], '24': ['23', '25'], '25': ['12', '24', '26'], '26': ['11', '25', '27'], '27': ['20', '26'], '3': ['2', '4'], '30': ['31', '37'], '31': ['30', '32', '46'], '32': ['31', '33', '45'], '33': ['32', '34'], '34': ['33', '35'], '35': ['34', '36'], '36': ['21', '35', '37'], '37': ['30', '36'], '4': ['3', '5'], '42': ['43'], '43': ['42', '44'], '44': ['43', '45'], '45': ['32', '44', '46'], '46': ['31', '45', '47'], '47': ['46'], '5': ['4', '6'], '6': ['5', '7'], '7': ['0', '6']}


Now we can submit a task of the above program with verbatim box.

In [137]:
verbatim_task = aspen_m1.run(OpenQASMProgram(source=program_with_verbatim_box), shots = 10)
verbatim_result = verbatim_task.result()
meta = verbatim_result.additional_metadata.rigettiMetadata
print(meta.compiledProgram)


DECLARE ro BIT[2]
PRAGMA INITIAL_REWIRING "NAIVE"
RX(pi) 0
RX(pi) 0
CZ 0 1
MEASURE 1 ro[1]
MEASURE 0 ro[0]



As shown above, the two consecutive `rx`  $\pi$-rotation gates did not get optimized and we confirm that our program was indeed executed verbatim.

## Requesting Result Types with OpenQASM

Braket provides [a rich library of result types](https://docs.aws.amazon.com/braket/latest/developerguide/braket-result-types.html) for circuit executions. With OpenQASM, requesting different result types for our tasks is easier than ever using the `result` pragma. Next, we give an example of requesting result types for our Bell state program submitted to SV1. Before doing that, let's see what result types are supported on SV1:

In [140]:
# print the result types supported by SV1
for iter in sv1.properties.action['braket.ir.openqasm.program'].supportedResultTypes:
    print(iter)

name='Sample' observables=['x', 'y', 'z', 'h', 'i', 'hermitian'] minShots=1 maxShots=100000
name='Expectation' observables=['x', 'y', 'z', 'h', 'i', 'hermitian'] minShots=0 maxShots=100000
name='Variance' observables=['x', 'y', 'z', 'h', 'i', 'hermitian'] minShots=0 maxShots=100000
name='Probability' observables=None minShots=1 maxShots=100000
name='Amplitude' observables=None minShots=0 maxShots=0


With knowing the supported result types on SV1, we choose to request the `Expectation` of $X \otimes Z$ observable on `q[0]` and `q[1]` and the `Amplitude` result type for a `shots=0` task of our bell program:

In [141]:
bell_with_result_type = """
OPENQASM 3;

qubit[2] q;

#pragma braket result expectation x(q[0]) @ z(q[1])
#pragma braket result amplitude "00", "11"
h q[0];
cnot q[0], q[1];
"""

The location of the `result` pragma is very flexible as long as it's after the qubit register definition (if you use physical qubits, you can put `result` pragmas anywhere after the program header).

We can submit the above program and receive the results for our requested result types.

In [142]:
bell_result_types_task = sv1.run(OpenQASMProgram(source=bell_with_result_type), shots = 0)
bell_result = bell_result_types_task.result()
values = bell_result.values
print(values)

[0.0, {'00': (0.7071067811865475+0j), '11': (0.7071067811865475+0j)}]


At last, we want to remind our Braket OpenQASM users that there are two requirements when requesting result types:
1. For `shots=0` tasks, requesting non-simultaneously measurable result types is allowed, but for `shots>0` tasks, it is not allowed. For example, we can write the following OpenQASM program in a `shots=0` task but not in a `shots>0` task, since the two result types are not simultaneously measurable:

In [ ]:
program_with_non_simultaneously_measurable_result_types = """
OPENQASM 3;

qubit[2] q;

h q[0];
cnot q[0], q[1];

#pragma braket result expectation x(q[0]) @ z(q[1])
#pragma braket result expectation hermitian([[0, -1im], [1im, 0]]) q[0]
"""

2. Do not use measurement instructions and request result types in the same OpenQASM program, otherwise a validation error will be raised. Since measurement instructions are basically equivalent to `#pragma braket result sample z(qubit)`, we encourage users to adapt a consistent style of requesting result types in the same program.

# Conclusion

In this notebook, you learned how to submit OpenQASM tasks and use OpenQASM features on Braket. Hope you enjoyed it! You can find more information about OpenQASM3.0 in its [live specification](https://qiskit.github.io/openqasm/index.html), and you can learn more about OpenQASM support on Braket in the [Amazon Braket documentation](https://docs.aws.amazon.com/braket/latest/developerguide/).